In [13]:
from gan_protein_structural_requirements.utils.protein_visualizer import jupy_viz_obj as jvo, jupy_viz_file as jvf
from gan_protein_structural_requirements.utils.folding_models import esm_batch_predict as esm_batch_predict, esm_predict as esm_predict
from gan_protein_structural_requirements.utils.extract_structure import extract_structures as extract_structures

In [14]:
data = extract_structures("./gan_protein_structural_requirements/data/raw/batch_1_data/")

/Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/Bio/PDB/DSSP.py:199: UserWarning: no atoms read before TER record 
TER    3208      PHE A 315                                                      

  warnings.warn(err)
/Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/Bio/PDB/DSSP.py:199: UserWarning: no atoms read before TER record 
TER    4101      VAL D  66                                                      

  warnings.warn(err)
/Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/Bio/PDB/DSSP.py:199: UserWarning: no atoms read before TER record 
TER    3208      PHE A 315                                                      

  warnings.warn(err)
/Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/Bio/PDB/DSSP.py:199: UserWarning: no atoms read before TER record 
TER    4101      VAL D  66                                                      

  warnings.warn(e

In [22]:
import pandas as pd
import numpy as np

arr = []

for entry in data["secondary"]:
    id = entry
    seq = data["secondary"][id]['sequence']
    c8 = data["secondary"][id]['c8']
    pol = data["primary_pol"][id]["polarity_conv"]
    arr.append([id, seq, c8, pol])

dat = np.array(arr)

df = pd.DataFrame(dat, columns=["id","seq","c8","polarity"])
df.head()

print(df.seq.map(lambda x: len(x)).max())

#max len of tokens = 1000

621
